In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
import qcodes.instrument_drivers.tektronix.AWG5014 as awg
awg1 = awg.Tektronix_AWG5014('AWG1', 'TCPIP0::172.20.3.248::inst0::INSTR', timeout=40)

Connected to: TEKTRONIX AWG5014C (serial:B051039, firmware:SCPI:99.0 FW:4.6.0.7) in 0.12s


In [3]:
# Note: if you have had any initialisation problems, the VISA message queue of the instrument could be fouled up
#(e.g. if the cell above gave you back something like 'Connected to: 1.20000000E+009 ...')
# in that case, run the following command to reset the message queue
awg1.clear_message_queue()

As with any other QCoDeS instrument, parameters can be set and get (gotten).

In [4]:
print(awg1.ch3_state.get())
print(awg1.ch2_offset.get())
awg1.ch2_offset.set(0.1)
print(awg1.ch2_offset.get())

0

0.1
0.1


A list of all available parameters can be found in the following manner:

In [5]:
pars = np.sort(list(awg1.parameters.keys()))
for param in pars:
    print(param, ': ', awg1.parameters[param].label)

IDN :  IDN
ch1_add_input :  Add input channel {}
ch1_amp :  Amplitude channel 1 (Vpp)
ch1_direct_output :  Direct output channel 1
ch1_filter :  Low pass filter channel 1
ch1_m1_del :  Channel 1 Marker 1 delay (ns)
ch1_m1_high :  Channel 1 Marker 1 high level (V)
ch1_m1_low :  Channel 1 Marker 1 low level (V)
ch1_m2_del :  Channel 1 Marker 2 delay (ns)
ch1_m2_high :  Channel 1 Marker 2 high level (V)
ch1_m2_low :  Channel 1 Marker 2 low level (V)
ch1_offset :  Offset channel 1 (V)
ch1_state :  Status channel 1
ch1_waveform :  Waveform channel 1
ch2_add_input :  Add input channel {}
ch2_amp :  Amplitude channel 2 (Vpp)
ch2_direct_output :  Direct output channel 2
ch2_filter :  Low pass filter channel 2
ch2_m1_del :  Channel 2 Marker 1 delay (ns)
ch2_m1_high :  Channel 2 Marker 1 high level (V)
ch2_m1_low :  Channel 2 Marker 1 low level (V)
ch2_m2_del :  Channel 2 Marker 2 delay (ns)
ch2_m2_high :  Channel 2 Marker 2 high level (V)
ch2_m2_low :  Channel 2 Marker 2 low level (V)
ch2_offse

## SENDING WAVEFORMS

There are two supported ways of sending waveforms to the AWG; by making an .awg file, sending and loading that, or by sending waveforms to the User Defined list one by one and putting them in the sequencer. The first method is generally faster.

### MAKING SOME WAVEFORMS

First, we make a handful of pulse shapes and marker signals to send to the AWG. This should be done with numpy arrays.

Please note that the waveforms must **not** have values exceeding -1 to 1, and that the markers must **only** have values 0 or 1. Otherwise the AWG misinterprets the signals.

In this example, we only use two channels of the AWG. Extending to one or more should be straightforward.

In [33]:
noofseqelems = 6
noofpoints = 1200
waveforms = [[], []]  # one list for each channel
m1s = [[], []]
m2s = [[], []]
for ii in range(noofseqelems):
    # waveform and markers for channel 1
    waveforms[0].append(np.sin(np.pi*(ii+1)*np.linspace(0, 1, noofpoints))*np.hanning(noofpoints))
    m1 = np.zeros(noofpoints)
    m1[:int(noofpoints/(ii+1))] = 1
    m1s[0].append(m1)
    m2 = np.zeros(noofpoints)
    m2s[0].append(m2)
    
    # waveform and markers for channel two
    wf = np.sin(np.pi*(ii+1)*np.linspace(0, 1, noofpoints))
    wf *= np.arctan(np.linspace(-20, 20, noofpoints))/np.pi*2
    waveforms[1].append(wf)
    m1 = np.zeros(noofpoints)
    m1[:int(noofpoints/(ii+1))] = 1
    m1s[1].append(m1)
    m2 = np.zeros(noofpoints)
    m2s[1].append(m2)

In [52]:
# We can visualise the waveforms and markers
fig = plt.figure()

ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412)
ax3 = fig.add_subplot(413)
ax4 = fig.add_subplot(414)
ax1.set_title('Channel 1 waveform')
ax1.set_ylim([-1.1, 1.1])
ax2.set_title('Channel 1 markers')
ax2.set_ylim([-0.1, 1.1])
ax3.set_title('Channel 2 waveform')
ax3.set_ylim([-1.1, 1.1])
ax4.set_title('Channel 2 markers')
ax4.set_ylim([-0.1, 1.1])

elemnum = 4  # choose which element to plot
ax1.plot(waveforms[0][elemnum], lw=4, color='#e1cb66')
ax2.plot(m1s[0][elemnum], lw=2, color='#FF4500', alpha=0.6)
ax2.plot(m2s[0][elemnum], lw=2, color='#FF8C00', alpha=0.6)
ax3.plot(waveforms[1][elemnum], lw=4, color='#6689e1')
ax4.plot(m1s[1][elemnum], lw=2, color='#6A5ACD', alpha=0.6)
ax4.plot(m2s[1][elemnum], lw=2, color='#EE82EE', alpha=0.6)

plt.tight_layout()

<IPython.core.display.Javascript object>

### GETTING THEM THERE VIA AN .AWG FILE

The fastest way to transfer waveforms to the AWG is by using an .awg file. 

In addition to waveforms and markers, me must specify sequencing options for each sequence element.

In this example notebook, we just put in some random numbers.

In [8]:
# Sequencing options

# number of repetitions
nreps = [2 for ii in range(noofseqelems)]
# Wait trigger (0 or 1)
trig_waits = [0]*noofseqelems
# Goto state
goto_states = [((ii+1) % noofseqelems)+1 for ii in range(noofseqelems)]
# Event jump
jump_tos = [2]*noofseqelems

Then all the magic happens in the following function call (type help(make_send_and_load_awg_file) to see all possible input parameters):

In [27]:
awg1.make_send_and_load_awg_file(waveforms, m1s, m2s, 
                                 nreps, trig_waits,
                                 goto_states, jump_tos)

### GETTING THEM THERE BY SENDING WAVEFORMS TO LIST

An alternative way to transfer waveforms to the AWG is by sending them directly to the waveform list and then putting them into the sequencer. This method is more explicit, but slower for large waveforms.

In [30]:
# First clear up the waveform list and empty the sequencer
awg1.delete_all_waveforms_from_list()
# Then set the sequence length to the correct number lest not all sequence elements get uploaded
awg1.sequence_length.set(noofseqelems)

In [31]:
#Then transfer the waveforms to the list...
for elnum in range(noofseqelems):
    for chnum in range(2):
        wfmname = 'wfm{:03d}ch{}'.format(elnum, chnum+1)
        awg1.send_waveform_to_list(waveforms[chnum][elnum], m1s[chnum][elnum], m2s[chnum][elnum], wfmname)
#...upload them to the sequencer...
for elnum in range(noofseqelems):
    for chnum in range(2):
        wfmname = 'wfm{:03d}ch{}'.format(elnum, chnum+1)
        awg1.set_sqel_waveform(wfmname, chnum+1, elnum+1)
#...and set the sequence elements setting
for elnum in range(noofseqelems):
    awg1.set_sqel_goto_target_index(elnum+1, goto_states[elnum])
    awg1.set_sqel_loopcnt(nreps[elnum], elnum+1)
    awg1.set_sqel_trigger_wait(elnum+1, trig_waits[elnum])
    awg1.set_sqel_event_target_index(elnum+1, jump_tos[elnum])

Finally, close down the instrument.

In [17]:
awg1.close()